In [ ]:
from bs4 import BeautifulSoup
from bs4 import NavigableString, Tag

import requests
import json
import os

def getInfoDic(bsSoup):
    valueDic ={}

    coverPath = bsSoup.find("img", class_="alignnone size-full")['src']
    fullTitle = bsSoup.find("h1", class_="h3-size entry-title").get_text()
    title = fullTitle[fullTitle.find(' ')+1:] 
    ID = fullTitle[0:fullTitle.find(' ')]
    valueDic['封面圖'] = coverPath
    valueDic['標題'] = title
    valueDic['序號'] = ID
    
    pInfo = bsSoup.find("div", class_="project_info")  
    p = pInfo.find('p')

    while p:
        if not p.has_attr('class'):      # most p does not have class
            if p.contents[0].has_attr('class') and p.contents[0]['class'] == ['header']:
                header = p.contents[0].get_text().strip(' :')
                if (len(p) == 2):
                    if isinstance(p.contents[1], NavigableString):
                        body = p.contents[1]
                    else:
                        body = p.contents[1].get_text()
                else:
                    body = ""
                valueDic[header] = body
                p = p.find_next_sibling('p')
        else:
            header = p.get_text().strip(' :')
            body = []
            p = p.find_next_sibling('p')
            if (p.contents[0]['class'] == ['genre']):
                 for child in p.contents[0]:
                    body.append(child.get_text())
            p = p.find_next_sibling('p')
            valueDic[header] = body

    return valueDic


def getSoupFromJavhoo(avID):
    url = "https://www.javhoo.com/av/" + avID.lower()
    r = requests.get(url)
    if r.status_code == 404:
        return False
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    return soup

def downloadImage(webPath, avID):
    ext = webPath[webPath.rfind('.'):]
    filename = localFolder + '/' + avID + ext
    r = requests.get(webPath, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)  

def dumpDic2Json(valueDic):
    ID = valueDic['序號']
    filename = localFolder + '/' + ID +'.json'
    f = open(filename, "w")
    json.dump(valueDic, f)
    f.close()
    
def loadDicFromJson(avID):
    filename = localFolder + '/' + avID +'.json'    
    f = open(filename, 'r')
    valueDic = json.load(f)
    f.close()
    return valueDic
    
def checkAvIdExist(avID):
    filename = localFolder + '/' + avID +'.json'
    if os.path.exists(filename):
        return True
    return False

def checkAvIdImgExist(avID):
    jpg = localFolder + '/' + avID +'.jpg'
    png = localFolder + '/' + avID +'.png'
    if os.path.exists(jpg) or os.path.exists(png):
        return True
    return False


def completeAvID(avID, showInfo = False):
    vDic = {}
    #AvId status, 1 not download before, 2 not exist on Javhoo, 3 downloaded
    if not checkAvIdExist(avID):
        soup = getSoupFromJavhoo(avID)
        if not soup:
            return False
        vDic = getInfoDic(soup)
        dumpDic2Json(vDic)
        if showInfo:
            print("Downloaded ", avID)
    else:
        vDic = loadDicFromJson(avID)

    # check if img exist, otherwise download img
    if not checkAvIdImgExist(avID):
        downloadImage(vDic['封面圖'], avID)
        if showInfo:
            print("Img Downloaded", avID)
            
    return True


def dumpResult2Json(avIDResult):
    filename = 'avIDResult.json'
    f = open(filename, "w")
    json.dump(avIDResult, f)
    f.close()
    
def loadResultFromJson():
    filename = 'avIDResult.json'    
    f = open(filename, 'r')
    avIDResult = json.load(f)
    f.close()
    return avIDResult

def countAVResult(avIDResult):
    count0 = 0
    count404 = 0
    count200 = 0
    for key, value in avIDResult.items():
        if value[0] == 0:
            count0+=1
        elif value[0] == 200:
            count200+=1
        elif value[0] == 404:
            count404+=1
    print("AVIDResult: 0~", count0, " 200~", count200, " 404~", count404)

from itertools import islice
from random import randint

def randomGet(avIDResult, count):
    try_batch = list(islice(avIDResult.items(), len(avIDResult)))

    x = [randint(0, len(avIDResult)) for p in range(0, count)]

    for i in x:
        item = try_batch[i]
        if item[1][0] == 0:    # not get from web before
            result = completeAvID(item[0])
            if result:
                item[1][0] = 200
                print(item[0], " found----")
            else:
                item[1][0] = 404
                print(item[0], " not on Javhoo@@@@")
        else:
            print(item[0], " already done!!!!")
        
    
localFolder = '/Users/yiwenliu/Desktop/Projects/AV'

In [ ]:
avIDResult = loadResultFromJson()

In [ ]:
randomGet(avIDResult, 50)
dumpResult2Json(avIDResult)

In [ ]:
countAVResult(avIDResult)